In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import statsmodels.api as sm
import itertools as it
import re
from collections import Counter
from statsmodels.sandbox.stats.multicomp import multipletests
from itertools import compress
from scipy.stats.kde import gaussian_kde
from numpy import linspace
import scipy
from scipy import stats

In [2]:
def read_epfile(directory, file_name):
    with open(directory + file_name) as f:
        crnannot = []
        for line in f:
            tmp = line.strip().split("\t")
            tmp[1] = int(tmp[1])
            tmp[2]= int(tmp[2])
            crnannot.append(tmp)
            #if tmp[5] not in sraid.keys():
               # sraid[tmp[5]] = [tmp[3]]
            #else:
            #    sraid[tmp[5]].append(tmp[3])
    return crnannot

In [3]:
def write_result(directory, out_list, out_name):
    with open(directory+out_name, 'a') as file1:
        for i in range(len(out_list)):
            file1.write('\t'.join(map(str, out_list[i])) + '\n')
    file1.close()

In [4]:
def plot_ecdf(x1, x2, dlab, clr1, clr2, tit, xlab, fig_name):
    fig, ax = plt.subplots(figsize=(8, 7))
    
    n1, bins1, patches1 = ax.hist(x1, bins = 1000, normed=1, histtype='step', cumulative=True, label=dlab[0], color = clr1, linewidth = 4)
    n2, bins2, patches2 = ax.hist(x2, bins = 1000, normed=1, histtype='step', cumulative=True, label=dlab[1], color = clr2, linewidth = 4)
    patches1[0].set_xy(patches1[0].get_xy()[:-1])
    patches2[0].set_xy(patches2[0].get_xy()[:-1])
    ax.legend(loc = 'lower right', fontsize = 18)
    plt.title(tit, fontsize = 18)
    plt.xlabel(xlab, fontsize = 18)
    plt.ylabel("Empirical Cum. Dist. Func. (ECDF)", fontsize = 18)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 18)
    plt.title(tit, fontsize = 18)
    #plt.savefig(fig_name+'.pdf', dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()   

In [5]:
def plot_2hist(x1, x2, bin_lims, lab1, lab2, clr1, clr2, tit, xlab, fig_name):
    bin_centers = 0.5*(bin_lims[:-1]+bin_lims[1:])
    bin_widths = bin_lims[1:]-bin_lims[:-1]
    hist1, _ = np.histogram(x1, bins=bin_lims)
    hist2, _ = np.histogram(x2, bins=bin_lims)
    
    ##normalizing
    hist1b = hist1/np.max(hist1)
    hist2b = hist2/np.max(hist2)

    fig, (ax2) = plt.subplots(nrows = 1, ncols = 1, figsize=(8, 6))

    ax2.bar(bin_centers, hist1b, width = bin_widths, align = 'center', label = lab1, color = clr1, alpha = 0.5)
    ax2.bar(bin_centers, hist2b, width = bin_widths, align = 'center', label = lab2, color = clr2, alpha = 0.5)
    ax2.legend(loc = 'upper right', fontsize = 18)    
    plt.title(tit, fontsize = 18)
    plt.xlabel(xlab, fontsize = 18)
    plt.ylabel("Relative Proportion", fontsize = 18)
    #plt.savefig(fig_name+'.pdf', dpi=300)
    plt.show()

In [6]:
def plot_boxplot(dataset, dlabel, clr, tit, ylab, fig_name):
    fig = plt.figure(figsize = (4,6))
    medianprops = dict(linewidth = 3, color=clr)
    i=0
    boxprops = dict(linewidth = 1.5)
    toplot = [np.asarray([]) for i in range(len(dataset))]
    for d in dataset:
        #medianprops = dict(linewidth = 3, color=colcode[i])
        datax = toplot
        datax[i] = np.asarray(dataset[i])
        plt.boxplot(datax, widths = 0.6, medianprops = medianprops, boxprops = boxprops)
        i +=1
    plt.xticks([i for i in range(1, len(dataset)+1)], dlabel, fontsize = 18)
    plt.yticks(fontsize = 18)
    plt.ylabel(ylab, fontsize = 18)
    plt.title(tit, fontsize = 18)
    #plt.savefig(fig_name+'.pdf', dpi=150, bbox_inches="tight")
    plt.show()
    plt.close()

In [7]:
def plot_aggregate(cnt1, cnt2, xlab, xaxisl, figdim, col1, col2, fig_name):
    ## plotting begins
    xlimit = len(xlab)
    x = np.arange(0,len(xlab))
    #fig = plt.figure(figsize = (12,5))
    fig = plt.figure(figsize = figdim)
    ax = fig.add_subplot(111)
    prob1 = [x/sum(cnt1) for x in cnt1]
    prob2 = [x/sum(cnt2) for x in cnt2]
    rects1 = ax.bar(x-0.2, prob1, width = 0.35, color = col1, align = 'center')
    rects2 = ax.bar(x+0.2, prob2, width = 0.35, color = 'grey', align = 'center')
    plt.plot(x, prob1, color = col2, marker = '^')
    plt.plot(x, prob2, color = 'black', marker = 'o')
    #plt.title(fig_title, fontsize = 17)
    plt.xlabel(xaxisl, fontsize = 16)
    plt.ylabel("Probability", fontsize = 16)
    plt.xticks(np.arange(0,xlimit, step = 1), xlab, fontsize = 15)
    plt.yticks(fontsize = 15)
    #plt.ylim(ymax=1)
    plt.ylim(ymax=max(max(prob1), max(prob2))+0.05)
    ax.legend((rects1[0], rects2[0]), ('Significant','Insignificant'), fontsize = 15)
    #plt.show()
    plt.savefig(fig_name+'.pdf', dpi=300, bbox_inches="tight")
    plt.close()

In [8]:
def read_bedfile(directory, file_name):
    with open(directory + file_name) as f:
        bed = {}
        for line in f:
            tmp = line.strip().split("\t")
            lpid = tmp[4]
            if lpid not in bed.keys():
                bed[lpid] = tmp
            else: # already seen
                if int(bed[lpid][-1]) < int(tmp[-1]): # replace if current overlap is larger than older
                    bed[lpid] =tmp
    return bed

In [48]:
directory='/Users/kimm/Desktop/GM12878_files/'
rnapii_linterf = 'LHG0035N_0035V_0045V.e500.clusters.cis.BE5.pksupport.bedpe.EP.linter.20200630.bed'
rnapii_rinterf = 'LHG0035N_0035V_0045V.e500.clusters.cis.BE5.pksupport.bedpe.EP.rinter.20200630.bed'
cohesin_linterf = 'LHG0051H_0104V.e500.clusters.cis.BE9.pksupport.bedpe.EP.linter.20200630.bed'
cohesin_rinterf = 'LHG0051H_0104V.e500.clusters.cis.BE9.pksupport.bedpe.EP.rinter.20200630.bed'
epfile = 'GM12878_ChIA-PET_no-CTCF_yes-cohesin_yes-RNAPII_yes-NIPBL_peaks_chromHMM_EP_RAIDannot_20200630.bed'

In [49]:
crnannot = read_epfile(directory, epfile)

In [50]:
rnapii_linter = read_bedfile(directory, rnapii_linterf)
rnapii_rinter = read_bedfile(directory, rnapii_rinterf)
cohesin_linter = read_bedfile(directory, cohesin_linterf)
cohesin_rinter = read_bedfile(directory, cohesin_rinterf)

In [55]:
rnapii_orig_loops = []
for key, val in rnapii_linter.items():
    linter = val
    rinter = rnapii_rinter[key]
    tmp = [linter[0], int(linter[1]), int(linter[2])]
    tmp.extend([rinter[0], int(rinter[1]), int(rinter[2]), int(rinter[3]), linter[4]])
    tmp.extend([linter[8], linter[9], linter[10]])
    tmp.extend([rinter[8], rinter[9], rinter[10]])
    rnapii_orig_loops.append(tmp)

In [56]:
cohesin_orig_loops = []
for key, val in cohesin_linter.items():
    linter = val
    rinter = cohesin_rinter[key]
    tmp = [linter[0], int(linter[1]), int(linter[2])]
    tmp.extend([rinter[0], int(rinter[1]), int(rinter[2]), int(rinter[3]), linter[4]])
    tmp.extend([linter[8], linter[9], linter[10]])
    tmp.extend([rinter[8], rinter[9], rinter[10]])
    cohesin_orig_loops.append(tmp)

In [18]:
print("RNAPII original loops: " + str(len(rnapii_orig_loops)))
print("Cohesin original loops: " + str(len(cohesin_orig_loops)))

RNAPII original loops: 147737
Cohesin original loops: 161911


In [28]:
write_result(directory, rnapii_orig_loops, 'LHG0035N_0035V_0045V.e500.clusters.cis.BE5.pksupport.EPannot.20200630.bedpe')

In [29]:
write_result(directory, cohesin_orig_loops, 'LHG0051H_0104V.e500.clusters.cis.BE9.pksupport.EPannot.20200630.bedpe')

In [57]:
rnapii_adj_loops = []
for key, val in rnapii_linter.items():
    linter = val
    rinter = rnapii_rinter[key]
    if linter[5] != '.' and rinter[5] != '.': # both anchors overlap E or P
        chrom = linter[0]
        lstart = max(int(linter[1]), int(linter[6]))
        lend = min(int(linter[2]), int(linter[7]))
        rstart = max(int(rinter[1]), int(rinter[6]))
        rend = min(int(rinter[2]), int(rinter[7]))
        tmp = [chrom, lstart, lend, chrom, rstart, rend, int(linter[3]), linter[4]]
        tmp.extend([linter[8], linter[9], linter[10]])
        tmp.extend([rinter[8], rinter[9], rinter[10]])
        rnapii_adj_loops.append(tmp)

In [58]:
cohesin_adj_loops = []
for key, val in cohesin_linter.items():
    linter = val
    rinter = cohesin_rinter[key]
    if linter[5] != '.' and rinter[5] != '.': # both anchors overlap E or P
        chrom = linter[0]
        lstart = max(int(linter[1]), int(linter[6]))
        lend = min(int(linter[2]), int(linter[7]))
        rstart = max(int(rinter[1]), int(rinter[6]))
        rend = min(int(rinter[2]), int(rinter[7]))
        tmp = [chrom, lstart, lend, chrom, rstart, rend, int(linter[3]), linter[4]]
        tmp.extend([linter[8], linter[9], linter[10]])
        tmp.extend([rinter[8], rinter[9], rinter[10]])
        cohesin_adj_loops.append(tmp)

In [59]:
print("RNAPII filtered loops: " + str(len(rnapii_adj_loops)))

RNAPII filtered loops: 12654


In [60]:
print("Cohesin filtered loops: " + str(len(cohesin_adj_loops)))

Cohesin filtered loops: 7978


In [34]:
write_result(directory, rnapii_adj_loops, 'LHG0035N_0035V_0045V.e500.clusters.cis.BE5.pksupport.EPonly.20200630.bedpe')

In [35]:
write_result(directory, cohesin_adj_loops, 'LHG0051H_0104V.e500.clusters.cis.BE9.pksupport.EPonly.20200630.bedpe')

In [61]:
raidcnt = Counter([x[4] for x in crnannot])

In [62]:
lp_dict = {}
for x in cohesin_adj_loops:
    if x[9] != '.' and raidcnt[x[9]] > 1: # RAID has more than 1 EP
        combid = x[9] + ";" + x[8]+","+x[11]
        if x[9] == x[12]: # same RAID
            if combid not in lp_dict.keys():
                lp_dict[combid] = [[x[6]], []]
            else:
                lp_dict[combid][0].append(x[6])

In [63]:
for x in rnapii_adj_loops:
    if x[9] != '.' and raidcnt[x[9]] > 1: # RAID has more than 1 EP
        combid = x[9] + ";" + x[8]+","+x[11]
        if x[9] == x[12]: # same RAID
            if combid not in lp_dict.keys():
                lp_dict[combid] = [[], [x[6]]]
            else:
                lp_dict[combid][1].append(x[6])

In [64]:
len(lp_dict)

4838

In [65]:
len([x for x in lp_dict.values() if len(x[1]) == 0 or len(x[0])==0])

1548

In [66]:
len([x for x in lp_dict.values() if len(x[1]) == 0])

608

In [67]:
max([len(x[0]) for x in lp_dict.values()])

36

In [88]:
lp_dict_raid = {}
for key, val in lp_dict.items():
    raidid = key.split(";")[0]
    if raidid not in lp_dict_raid.keys():
        lp_dict_raid[raidid] = [sum(val[0]), sum(val[1])] # cohesin, rnapii
    else:
        lp_dict_raid[raidid][0] += sum(val[0]) # cohesin
        lp_dict_raid[raidid][1] += sum(val[1]) # rnapii

In [70]:
del cohesin_concat
del rnapii_concat
del cbed

In [71]:
cohesin_concat = []
rnapii_concat = []
cohesin_lpcnt = []
cohesin_petcnt = []
rnapii_lpcnt = []
rnapii_petcnt = []
for key,val in lp_dict.items():
    #if len(val[0]) != 0 and len(val[1]) != 0:
    cohesin_lpcnt.append(len(val[0]))
    cohesin_petcnt.append(sum(val[0]))
    rnapii_lpcnt.append(len(val[1]))
    rnapii_petcnt.append(sum(val[1]))
    lrid = key.split(";")[1].split(",")
    left = [x for x in crnannot if x[3]==lrid[0]][0]
    right = [x for x in crnannot if x[3]==lrid[1]][0]
    cbed = [left[0], left[1], left[2]]
    cbed.extend([right[0], right[1], right[2]])
    cbed.append(sum(val[0]))
    cbed.extend([left[3], left[4], left[5], right[3], right[4], right[5]])
    if lrid[0] == lrid[1]: # if loop within same peak
        midpoint = int((cbed[1]+cbed[2])/2)
        cbed[2] = midpoint
        cbed[4] = midpoint
    cohesin_concat.append(cbed)
    ## RNAPII
    rbed = [left[0], left[1], left[2]]
    rbed.extend([right[0], right[1], right[2]])
    rbed.append(sum(val[1]))
    rbed.extend([left[3], left[4], left[5], right[3], right[4], right[5]])
    if lrid[0] == lrid[1]: # if loop within same peak
        midpoint = int((rbed[1]+rbed[2])/2)
        rbed[2] = midpoint
        rbed[4] = midpoint
    rnapii_concat.append(rbed)

In [72]:
len(rnapii_concat)

4838

In [73]:
len(cohesin_concat)

4838

In [53]:
write_result(directory, rnapii_concat, 'LHG0035N_0035V_0045V.e500.clusters.cis.BE5.pksupport.EPonly.inRAID.concat.20200630.bedpe')

In [54]:
write_result(directory, cohesin_concat, 'LHG0051H_0104V.e500.clusters.cis.BE9.pksupport.EPonly.inRAID.concat.20200630.bedpe')

In [74]:
comb_byraid = {}
for x in rnapii_concat:
    raidid = x[8]
    if raidid not in comb_byraid.keys():
        comb_byraid[raidid] = [x]
    else:
        comb_byraid[raidid].append(x)

In [75]:
len(comb_byraid)

609

In [76]:
print(sum([len(val) for key, val in comb_byraid.items()]))

4838


In [77]:
print(len([x for x in rnapii_concat if (x[4]-x[2]) > 100000]))

1706


In [78]:
comb_byraid_G100kb = {}
comb_byraid_G50kb = {}
raid_G100kb = []
raid_G50kb = []
for key, val in comb_byraid.items():
    filtered100kb = []
    filtered50kb = []
    for x in val:
        if x[4]-x[2] > 100000:
            filtered100kb.append([x[0], x[1], x[2], x[3], x[4], x[5], x[7], x[10], x[8]+'-100kb'])
        if x[4]-x[2] > 50000:
            filtered50kb.append([x[0], x[1], x[2], x[3], x[4], x[5], x[7], x[10], x[8]+'-50kb'])
    if len(filtered100kb) > 0:
        comb_byraid_G100kb[key] = filtered100kb
        start = min([x[1]-x[5]+x[2] for x in filtered100kb])
        end = max([x[5]+x[4]-x[1] for x in filtered100kb])
        raid_G100kb.append([x[0], start, end, key+'-100kb', len(filtered100kb)])
    if len(filtered50kb) > 0:
        comb_byraid_G50kb[key] = filtered50kb
        start = min([x[1]-x[5]+x[2] for x in filtered50kb])
        end = max([x[5]+x[4]-x[1] for x in filtered50kb])
        raid_G50kb.append([x[0], start, end, key+'-50kb', len(filtered50kb)])

In [79]:
loops_G100kb = [item for sublist in comb_byraid_G100kb.values() for item in sublist]
loops_G50kb = [item for sublist in comb_byraid_G50kb.values() for item in sublist]

In [80]:
len(loops_G100kb)

1706

In [81]:
len(loops_G50kb)

2560

In [82]:
write_result(directory, loops_G100kb, 'CRN-loops.G100kb.20200705.bedpe')

In [83]:
write_result(directory, loops_G50kb, 'CRN-loops.G50kb.20200705.bedpe')

In [84]:
write_result(directory, raid_G100kb, 'CRN-loops.G100kb.RAID-annot.20200705.bed')

In [85]:
write_result(directory, raid_G50kb, 'CRN-loops.G50kb.RAID-annot.20200705.bed')